<a href="https://colab.research.google.com/github/darkmochalover/DS_TermProject_AL2/blob/main/logistic_regression_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dictionary

In [1]:
# Data Path
raw_data_path = "/content/drive/MyDrive/DS_TermProject/data/combined_mbti_df.csv"
# data_path = "data/preprocessed_mbti_data.csv"


audio_qualities = [
    'danceability_mean',
    'valence_mean',
    'energy_mean',
    'loudness_mean',
    'acousticness_mean',
    'instrumentalness_mean',
    'liveness_mean',
]

# 장조/단조 (Major/Minor)
all_tones = [
    'Cminor_count', 'CMajor_count', 'C#/Dbminor_count', 'C#/DbMajor_count',
    'DMajor_count', 'D#_EbMajor_count', 'Eminor_count', 'EMajor_count',
    'Fminor_count', 'FMajor_count', 'F#/Gbminor_count', 'GMajor_count',
    'G#/Abminor_count', 'G#/AbMajor_count', 'Aminor_count', 'AMajor_count',
    'A#/Bbminor_count', 'BMajor_count', 'Dminor_count', 'D#_Ebminor_count',
    'Gminor_count', 'A#/BbMajor_count', 'F#/GbMajor_count', 'Bminor_count'
]

major_tones = [
    'CMajor_count', 'C#/DbMajor_count',
    'DMajor_count', 'D#_EbMajor_count', 
    'EMajor_count',
    'FMajor_count', 
    'GMajor_count', 'G#/AbMajor_count', 
    'AMajor_count', 'BMajor_count', 'A#/BbMajor_count', 
    'F#/GbMajor_count'
]
minor_tones = [
    'Cminor_count', 'C#/Dbminor_count', 
    'Eminor_count', 
    'Fminor_count', 'F#/Gbminor_count', 
    'G#/Abminor_count',  
    'Aminor_count', 'A#/Bbminor_count', 
    'Dminor_count', 'D#_Ebminor_count',
    'Gminor_count', 
    'Bminor_count'
]


# 열 이름을 바꾸기 위해 리스트로 저장함
renamed_columns =  [
    'danceability',
    'valence',
    'energy',
    'loudness',
    'acousticness',
    'instrumentalness',
    'liveness'
]


Install WandB

In [2]:
!pip install -qU wandb

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from imblearn.over_sampling import SMOTE

import itertools

import wandb
import random

Step 1: Import W&B and Login

In [4]:
import wandb
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint, WandbEvalCallback

In [5]:
encoder_list = ['LabelEncoder','OneHotEncoder']
scaler_list = ['StandardScaler', 'MinMaxScaler', 'RobustScaler', 'No Scale']
test_size_list = [0.3, 0.2, 0.1]
do_smote = ['True', 'False']

In [6]:
config = {
        'encoder' : 'LabelEncoder',
        'scaler' : 'StandardScaler',
        'test_size' : 0.3,
        "do_smote" : True
    }

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="regression",
    
    # track hyperparameters and run metadata
    config = config
)

config=wandb.config


wandb: Currently logged in as: real100percent. Use `wandb login --relogin` to force relogin


Data Loading

In [7]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

raw_data_path = '/content/drive/MyDrive/DS_TermProject/data/combined_mbti_df.csv'
df = pd.read_csv(raw_data_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Feature Engineering

In [8]:
# Subset only measures of centers
X = df.iloc[: , :22] # 처음 22개 열을 선택해서 저장
X = df[audio_qualities] # 오디오 품질과 해당 열이 있는 열을 선택해서 저장


categories = renamed_columns[:]
X.columns = renamed_columns


# 장조/단조의 개수의 합을 계산해서 저장 (C장조, D단조, .. 이렇게 따로 계산되는거 말고, 위에 지정된 list 이용해서 sum값 넣어줌)
X['major_count'] = df[major_tones].sum(axis=1).astype('int64')
X['minor_count'] = df[minor_tones].sum(axis=1).astype('int64')

<ipython-input-8-482ca4c8bc7c>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['major_count'] = df[major_tones].sum(axis=1).astype('int64')
<ipython-input-8-482ca4c8bc7c>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['minor_count'] = df[minor_tones].sum(axis=1).astype('int64')


## Encoding & Scaling Function

In [9]:
def Encoding(df, encoding_method):
    df = df.copy()

    if(encoding_method == 'LabelEncoder'):
        encoder = LabelEncoder()
        target = encoder.fit_transform(df[['mbti']])
        

    if(encoding_method == 'OneHotEncoder'):
        encoder = OneHotEncoder(sparse=False)
        target = encoder.fit_transform(df[['mbti']])

    return target

def Scaling(scale_method, X_train, X_test):
    if( scale_method == 'No Scale'):
        return X_train, X_test

    elif(scale_method == 'StandardScaler'):
        scaler = StandardScaler()

    elif(scale_method == 'MinMaxScaler'):
        scaler = MinMaxScaler()

    elif(scale_method == 'RobustScaler'):
        scaler = RobustScaler()


    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.fit_transform(X_test)

    return X_train_scaled, X_test_scaled


In [10]:
# # If X and y have different shapes, adjust them accordingly
# if X.shape[0] > y.shape[0]:
#     X = X[:y.shape[0], :]
# else:
#     y = y[:X.shape[0]]

# # Print the adjusted shape of X and y
# print("Adjusted X shape:", X.shape)
# print("Adjusted y shape:", y.shape)

### Logistic Regression Hyperparameter tuning

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, recall_score, precision_score
import itertools
import wandb
import numpy as np

from sklearn.model_selection import GridSearchCV


# Define the combinations of hyperparameters
encoder_list = ['LabelEncoder', 'OneHotEncoder']
scaler_list = ['StandardScaler', 'MinMaxScaler', 'RobustScaler', 'No Scale']
test_size_list = [0.3, 0.2, 0.1]
do_smote_list = [True, False]
SearchMethods = ['Grid']
cv_list = [3, 5, 7]

# Parameter grid
param_grid = [    
    {'penalty' : ['l2', 'none'],
    'C' : [ 10, 1.0, 0.1],
    'solver' : ['lbfgs','newton-cg','liblinear'],
    'max_iter' : [100, 1000, 2500, 5000]
    }
]

combinations = list(itertools.product(encoder_list, scaler_list, test_size_list, do_smote_list, SearchMethods, cv_list))

for encoder, scaler, test_size, do_smote, search_method, cv in combinations:
    # Print the hyperparameter settings
    print(f"Encoder: {encoder}, Scaler: {scaler}, Test Size: {test_size}, SMOTE: {do_smote}")
    wandb.init(project='logistic_regression_final', config={
        'encoder': encoder,
        'scaler': scaler,
        'test_size': test_size,
        'do_smote': do_smote,
        'search_method' : search_method,
        "cv" : cv
    })

    
    # Apply the specified encoder and scaler
    config = wandb.config
    wandb.config.update(config)
    print(config)

    y = Encoding(df = df, encoding_method=encoder)

    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=5, test_size=test_size)


    X_train, X_test = Scaling(scale_method=scaler, X_train=X_train, X_test=X_test)

    if(do_smote == 'True'): # If true,
        smote = SMOTE(sampling_strategy='auto', random_state=0)
        X_train, y_train = smote.fit_resample(X_train,y_train)

    
    # 모델 평가 지표
    wandb.define_metric('accuracy', summary='max')
    wandb.define_metric('F1 Score', summary='max')
    wandb.define_metric('Recall', summary='max')
    wandb.define_metric('Precision', summary='max')


    # Create and fit the logistic regression model
    base_model = LogisticRegression(random_state=42)

    model = base_model.fit(X_train, y_train)
    predictions = base_model.predict(X_test)

    if(search_method == 'Base'):
      pass

    elif(search_method == 'Grid'):
      # Grid Search
      grid_search = GridSearchCV(estimator=base_model, param_grid = param_grid, cv = cv, verbose=True, n_jobs=-1)


      best_model = grid_search.fit(X_train, y_train)
      print(best_model.best_estimator_)

      best_params_grid = grid_search.best_params_
      print(best_params_grid)
      # {'C': 1.0, 'max_iter': 100, 'penalty': 'l2', 'solver': 'liblinear'}

      print("best_params_grid['C']: ", best_params_grid['C'])
      print("best_params_grid['max_iter']: ", best_params_grid['max_iter'])
      print("best_params_grid['penalty']:", best_params_grid['penalty'])
      print("best_params_grid['solver']:", best_params_grid['solver'])


      config.update({'C' : best_params_grid['C']})
      config.update({'max_iter' : best_params_grid['max_iter']})
      config.update({'penalty' : best_params_grid['penalty']})
      config.update({'solver' : best_params_grid['solver']})

      predictions = best_model.predict(X_test)

    

    # Calculate evaluation metrics
    accuracy = accuracy_score(y_test, predictions)
    cm = confusion_matrix(y_test, predictions)
    if(encoder == 'OneHotEncoder'):
        cm = confusion_matrix(y_test.argmax(axis=1), predictions.argmax(axis=1))
    else:
        cm = confusion_matrix(y_test, predictions)
    f1 = f1_score(y_test, predictions, average='macro')
    recall = recall_score(y_test, predictions, average='macro')
    precision = precision_score(y_test, predictions, average='macro')

    # Log the evaluation metrics to WandB
    wandb.log({'Accuracy': accuracy})
    wandb.log({'Confusion Matrix': cm})
    wandb.log({'F1 Score': f1})
    wandb.log({'Recall': recall})
    wandb.log({'Precision': precision})

    print(config)

    # Print the evaluation metrics
    print("Accuracy:", accuracy)
    print("Confusion Matrix:\n", cm)
    print("F1 Score:", f1)
    print("Recall:", recall)
    print("Precision:", precision)


Encoder: LabelEncoder, Scaler: StandardScaler, Test Size: 0.3, SMOTE: True


{'encoder': 'LabelEncoder', 'scaler': 'StandardScaler', 'test_size': 0.3, 'do_smote': True, 'search_method': 'Grid', 'cv': 3}


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Fitting 3 folds for each of 72 candidates, totalling 216 fits
LogisticRegression(random_state=42, solver='liblinear')
{'C': 1.0, 'max_iter': 100, 'penalty': 'l2', 'solver': 'liblinear'}
best_params_grid['C']:  1.0
best_params_grid['max_iter']:  100
best_params_grid['penalty']: l2
best_params_grid['solver']: liblinear
{'encoder': 'LabelEncoder', 'scaler': 'StandardScaler', 'test_size': 0.3, 'do_smote': True, 'search_method': 'Grid', 'cv': 3, 'C': 1.0, 'max_iter': 100, 'penalty': 'l2', 'solver': 'liblinear'}
Accuracy: 0.22285714285714286
Confusion Matrix:
 [[20 15  7  1  0  2  0  3  0  7  3  4  4  9  0  3]
 [ 7 31  7  6  0 11  0  5  2  2  3  1  2  4  0  2]
 [ 3  2 37 14  0  3  0  8  2  1  7  0  0  2  0  7]
 [ 3  6 14 39  0  7  0  7  1  2  3  4  0  1  0  7]
 [ 3  8  3  2  0  1  0  1  1  4  1  0  4  3  0  1]
 [ 5 20  8  4  1  6  0 10  2  4  2  1  3  2  0  0]
 [ 2  8 15  2  0  0  0  3  2  0  1  3  2  2  0  1]
 [ 5  9 18 17  0  7  0 19  2  0  9  2  2  2  0  7]
 [ 5  8  0  1  0  1  0  0 23 15

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
36 fits failed out of a total of 216.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
36 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 71, in _check_solver
    raise ValueError("penalty='none' i

...

LogisticRegression(C=10, penalty='none', random_state=42)
{'C': 10, 'max_iter': 100, 'penalty': 'none', 'solver': 'lbfgs'}
best_params_grid['C']:  10
best_params_grid['max_iter']:  100
best_params_grid['penalty']: none
best_params_grid['solver']: lbfgs
{'encoder': 'LabelEncoder', 'scaler': 'StandardScaler', 'test_size': 0.2, 'do_smote': False, 'search_method': 'Grid', 'cv': 5, 'C': 10, 'max_iter': 100, 'penalty': 'none', 'solver': 'lbfgs'}
Accuracy: 0.2178702570379437
Confusion Matrix:
 [[12  6  4  2  0  2  0  3  1  1  0  5  4  5  0  2]
 [ 6 18  3  5  0  5  0  5  0  2  3  0  1  3  0  1]
 [ 1  0 23 10  0  3  0  7  2  1  4  0  0  2  0  3]
 [ 3  4  7 26  0  2  0  8  1  1  2  3  0  0  0  5]
 [ 5  6  0  1  0  0  0  0  0  0  1  0  3  1  0  1]
 [ 4 13  3  7  1  2  0 10  1  1  2  0  2  2  0  0]
 [ 2  6  5  3  0  0  0  2  2  1  0  2  0  2  1  1]
 [ 4  7 10 14  0  2  0 18  1  0  6  1  1  1  0  3]
 [ 3  3  0  1  0  1  0  0 16 16 10  4  5  3  1  2]
 [10  6  1  3  0  0  1  2 13  9  4  3  5  0  0  1

Accuracy,▁
F1 Score,▁
Precision,▁
Recall,▁
Accuracy,0.21787


{'encoder': 'LabelEncoder', 'scaler': 'StandardScaler', 'test_size': 0.2, 'do_smote': False, 'search_method': 'Grid', 'cv': 7}


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Fitting 7 folds for each of 72 candidates, totalling 504 fits


In [12]:
wandb.finish()